In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath

from jwlab.cluster_analysis_perm import cluster_analysis_procedure
from jwlab.ml_prep_perm import prep_ml, slide_df, init, load_ml_data, get_bad_trials, map_participants,average_trials_and_participants
from jwlab.bad_trials import get_bad_trials, get_left_trial_each_word


from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, RepeatedKFold
from scipy import stats
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt


In [48]:
age_group = 11
useRandomizedLabel = False
averaging = "permutation"
sliding_window_config = [-200, 1000, [100], 100]
downsample_num=1000

In [49]:
X, y, good_trial_count, num_win = prep_ml(age_group, useRandomizedLabel, averaging, sliding_window_config, downsample_num=1000)

In [ ]:
X

#### Train and predict windows within an age group

In [52]:
#time val

from sklearn.svm import SVC, LinearSVC
win_size= 100
res = {}
ind = np.arange(-200, 1000, win_size).tolist()
cols = np.arange(-200, 1000, win_size).tolist()
time_table = pd.DataFrame(index=ind, columns=cols)
if len(num_win) > 1:
    print("Error: Function not supported for mutliple window lengths.")
else: 
    for i in range(num_win[0]):
        for j in range(num_win[0]): 
            train_win = (i * win_size) - 200
            test_win = (j * win_size) - 200
            X_train = X[0][i]
            y_train = y[0][i]
            X_test = X[0][j]
            y_test = y[0][j]
            
            model = LinearSVC(C=1e-9, max_iter=1000)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            testScore = accuracy_score(y_test,y_pred) 
            
            res[train_win, test_win] = testScore
            time_table.loc[train_win, test_win]=testScore
            
time_table.to_csv('12m_Timewin_Pred_100ms.csv')       

#### Train and predict windows between age groups

In [2]:
# 9m training windows
age_group = 9
useRandomizedLabel = False
averaging = "permutation"
sliding_window_config = [-200, 1000, [100], 100]
downsample_num=1000

X, y, good_trial_count, num_win = prep_ml(age_group, useRandomizedLabel, averaging, sliding_window_config, downsample_num=1000)

In [3]:
X_train = X
y_train = y

In [4]:
# 12m testing windows
age_group = 11
useRandomizedLabel = False
averaging = "permutation"
sliding_window_config = [-200, 1000, [100], 100]
downsample_num=1000

X_test, y_test, good_trial_count, num_win = prep_ml(age_group, useRandomizedLabel, averaging, sliding_window_config, downsample_num=1000)

In [5]:
#time val

from sklearn.svm import SVC, LinearSVC
win_size= 100
res = {}
ind = np.arange(-200, 1000, win_size).tolist()
cols = np.arange(-200, 1000, win_size).tolist()
time_table = pd.DataFrame(index=ind, columns=cols)
if len(num_win) > 1:
    print("Error: Function not supported for mutliple window lengths.")
else: 
    for i in range(num_win[0]):
        for j in range(num_win[0]): 
            train_win = (i * win_size) - 200
            test_win = (j * win_size) - 200
            X_train_i = X_train[0][i]
            y_train_i = y_train[0][i]
            X_test_j = X_test[0][j]
            y_test_j = y_test[0][j]
            
            model = LinearSVC(C=1e-9, max_iter=1000)
            model.fit(X_train_i, y_train_i)
            y_pred = model.predict(X_test_j)
            testScore = accuracy_score(y_test_j,y_pred) 
            
            res[train_win, test_win] = testScore
            time_table.loc[train_win, test_win]=testScore
            
time_table.to_csv('9mSubsetTrain_12mTest_Timewin_Pred_100ms.csv')       

In [8]:
ind = np.arange(-200, 1000, win_size).tolist()
cols = np.arange(-200, 1000, win_size).tolist()
time_table = pd.DataFrame(index=ind, columns=cols)
time_table.loc[100, 400]=99
time_table

,-200,-100,0,100,200,300,400,500,600,700,800,900
-200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,99,NaN,NaN,NaN,NaN,NaN
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# For null distribution
# MAKE SURE YOU TURN OFF PRINT FUNCTION

itr = 50 
arrTmass = []

for i in range(itr):
    tmass = cluster_analysis_procedure(11, True, "permutation", [-200, 1000, [10], 10], [3, 15, 20])
    arrTmass.append(round(tmass, 4))
    print(i)
print(arrTmass)

549.4334729161642
0


KeyboardInterrupt: 

In [ ]:


plt.hist(arrTmass, bins = 20)
plt.show()